## Functions

In [244]:
import LyndonWords
import importlib
importlib.reload(LyndonWords)
from LyndonWords import *
import itertools
from functools import partial
import multiprocessing as mp
from tqdm.notebook import tqdm
import math
import numpy as np
def genPermutations(type,n,k):
    with mp.Pool() as pool:
        for i in tqdm(pool.imap_unordered(partial(rootSystem,type=type,k=k),itertools.permutations(np.array(range(n+1)))),total=math.factorial(n+1)):
            yield i

In [245]:
def monotonicityConj(rootsys:rootSystem,wordToCheck:word):
    wordPlusDelta = rootsys.getWords(wordToCheck.weights + wordToCheck.delta)[0]
    parsedFormat = rootsys.parseToDeltaFormat(wordPlusDelta)
    if(type(parsedFormat[0]) is list):
        return True
    return False
#TODO: Updated bases on new claim on monotonicity
def checkMonotoneIncreasingImpCond(rootSystemPermutations):
    for rootsys in rootSystemPermutations:
        returnArr = []
        deltaWords = rootsys.getWords(rootsys.delta)
        temp = rootsys.checkMonotonicity("Increasing")
        monotoneIncreasing = [rootsys.getWords(i[0] + rootsys.delta)[0] for i in temp]
        deltaArrays = [i.string for i in deltaWords]
        for i in monotoneIncreasing:
            flag = False
            for j in deltaWords:
                if(not str(i).startswith(str(j)) and (i.weights - rootsys.delta)[rootsys.ordering[0].rootIndex] > 0):
                    flag = True
                    break
            if(flag):
                continue
            returnArr.append((str(rootsys.ordering),(i.weights - rootsys.delta)))
    return returnArr

In [246]:
def checkCondImpMonotoneIncreasing(rootSystemPermutations):
    returnArr = []
    for rootsys in rootSystemPermutations:
        deltaWords = rootsys.getWords(rootsys.delta)
        deltaArrays = [i.string for i in deltaWords]
        for base in rootsys.baseWeights[:-1]:
            secondWord = rootsys.getWords(base + rootsys.delta)[0]
            if(base[rootsys.ordering[0].rootIndex] == 0):
                continue
            deltaStart = False
            for i in deltaArrays:
                if(np.all(secondWord[:rootsys.deltaHeight] == i)):
                    deltaStart = True
                    break
            if deltaStart:
                continue
            if(rootsys.getMonotonicity(base) != 1):
                returnArr.append((str(rootsys.ordering),str(rootsys.getWords(base)[0])))
    return returnArr

In [247]:
def printByLength(rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.getWords(comb):
                print(" "+str(i))

In [248]:
def printByBaseWord(rootSystem:rootSystem,comb):
    k=0
    isImaginary = rootSystem.isImaginary(comb)
    for i in rootSystem.getAffineWords(comb):
        if(isImaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not isImaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [249]:
def exportByLength(rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootSystem.ordering)+"\n")
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseWeights:
            comb = i + rootSystem.delta * k
            for i in rootSystem.getWords(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

### Delta Pattern Detection

In [250]:
def deltaSplitAtLast(rootsys:rootSystem,index: int) -> bool:
    baseWord = rootsys.getWords(rootsys.delta)[index]
    deltaWords = rootsys.getAffineWords(rootsys.delta)[index + rootsys.n::rootsys.n]
    for deltaWord in deltaWords:
        parsed = rootsys.parseToDeltaFormat(deltaWord)
        if(len(parsed) != 3):
            return False
        if(parsed[2] != str(baseWord[-1])):
            return False
    return True

def deltaSplitAtCofac(rootsys: rootSystem,index: int) -> bool:
    baseWord = rootsys.getWords(rootsys.delta)[index]
    cofacSplit = rootsys.costfac(baseWord)[0].height
    for deltaWord in rootsys.getAffineWords(rootsys.delta)[index+rootsys.n::rootsys.n]:
        parsed = rootsys.parseToDeltaFormat(deltaWord)
        if(len(parsed) != 3):
            return False
        if(parsed[0] != baseWord.noCommas()[:cofacSplit] or parsed[2] != baseWord.noCommas()[cofacSplit:]):
            return False
    return True

def k3StartDeltaPattern(rootsys: rootSystem, index: int) ->bool:
    deltaWords = rootsys.getAffineWords(rootsys.delta)[index::rootsys.n]
    splitting = [i.noCommas() for i in rootsys.costfac(deltaWords[1])]
    if(len(rootsys.parseToDeltaFormat(deltaWords[1])) != 1):
        return False
    for i in deltaWords[2:]:
        parsed = rootsys.parseToDeltaFormat(i)
        if(len(parsed) != 3):
            return False
        if(parsed[0] != splitting[0] or parsed[2] != splitting[1] or type(parsed[1]) is not list):
            return False
    return True

def lastSmallestDeltaPattern(rootsys:rootSystem, index: int) -> bool:
    deltaWords = rootsys.getAffineWords(rootsys.delta)[index::rootsys.n]
    for i in range(-1,-rootsys.deltaHeight-1,-1):
        if(deltaWords[0][i].rootIndex == rootsys.ordering[0].rootIndex):
            smallestIndex = rootsys.deltaHeight + i
            break
    for deltaWord in deltaWords[2:]:
        parsed = rootsys.parseToDeltaFormat(deltaWord)
        if(len(parsed) != 3):
            return False
        if(smallestIndex != len(parsed[0])):
            return False
    return True

def twoDeltaWordsDeltaPattern(rootsys:rootSystem,index:int) -> bool:
    deltaWords = rootsys.getAffineWords(rootsys.delta)[index+rootsys.n*2::rootsys.n]
    for deltaWord in deltaWords:
        parsed = rootsys.parseToDeltaFormat(deltaWord)
        if(len(parsed) != 4):
            return False
        if(type(parsed[0]) != str or type(parsed[1]) != list or type(parsed[2]) != list or type(parsed[3])!= str):
            return False
    return True

In [251]:
class deltaTypes:
    def __init__(self,index:int,hs:list,type:str,insertedIndex:int,leftfac:str,rightfac:str):
        self.index = index
        self.hs = hs
        #is either "last" or "cofac" or "neither" if something goes horribly wrong
        self.type = type
        self.insertedIndex = insertedIndex
        self.leftfac = leftfac
        self.rightfac = rightfac
    def toList(self):
        return [self.index,self.type,self.insertedIndex,self.leftfac,self.rightfac]
class deltaTypesCollection:
    def __init__(self,rootsys:rootSystem,deltaTypesList):
        self.type = rootsys.type
        self.ordering = str(rootsys.ordering)
        self.deltaTypes = deltaTypesList
    def toCsv(self)->str:
        retstr = f"{self.type},{self.ordering},,,,\n"
        for i in self.deltaTypes:
            retstr+= f"{i.index},[{' '.join([str(j) for j in i.hs])}],{i.type},{i.insertedIndex},{i.leftfac},{i.rightfac}\n"
        return retstr
    def notAllLast(self) -> bool:
        for i in self.deltaTypes:
            if i.type != "last":
                return True
        return False

In [252]:
def GenerateDeltaTypes(rootsys:rootSystem,k=3) ->deltaTypesCollection:
    deltaWords = rootsys.getWords(rootsys.delta)
    listOfDeltaTypes = []
    kdeltaWords = rootsys.getWords(rootsys.delta*k)
    for i in range(rootsys.n):
        kdeltaWord = kdeltaWords[i]
        splitting = 0
        parsedForm = rootsys.parseToDeltaFormat(kdeltaWord)
        for j in parsedForm:
            if(type(j) is not str):
                splitting = j[0]
                break
        if(deltaSplitAtLast(rootsys,i)):
            breakType = "last"
        elif(deltaSplitAtCofac(rootsys,i)):
            breakType = "cofac"
        elif(k3StartDeltaPattern(rootsys,i)):
            breakType = "k3start"
        elif(lastSmallestDeltaPattern(rootsys,i)):
            breakType = "lastSmallest"
        elif(twoDeltaWordsDeltaPattern(rootsys,i)):
            breakType = "twoDeltaWords"
        else:
            breakType = "other"
        factors = rootsys.costfac(deltaWords[i])
        listOfDeltaTypes.append(deltaTypes(i+1,deltaWords[i].hs,breakType,splitting,factors[0].noCommas(),factors[1].noCommas()))
    return deltaTypesCollection(rootsys,listOfDeltaTypes)
        

In [253]:
def costfacDeltaTypeConditionsMet(rootsys,deltaWords,index):
    factorization = rootsys.costfac(deltaWords[index])
    for i in range(index):
        if(factorization[1] < deltaWords[i]):
            if(rootsys.eBracket(rootsys.costfac(deltaWords[index])[1] + deltaWords[i])):
                return True
        else:
            break
    minusarr = np.zeros(rootsys.n+1,dtype=int)
    minusarr[deltaWords[index][-1].rootIndex] += 1
    if(word.letterListCmp(rootsys.getWords(rootsys.delta - minusarr)[0].string,deltaWords[index][:-1]) != 0):
        return True
    for j in range(index):
        if(word.letterListCmp(rootsys.costfac(deltaWords[j])[0].string,
                        rootsys.costfac(deltaWords[index])[1].string[:-1]) == 0):
            return True
    return False
def lastSplitDeltaTypeConditionsMet(rootsys:rootSystem,deltaWords:list,index:int) -> bool:
    if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
        return True
    rightfac = rootsys.costfac(deltaWords[index])[1]
    for deltaWord in deltaWords[:index]:
        if(word.letterListCmp(rootsys.costfac(deltaWord)[1].string,rightfac.string[:-1]) == 0):
            return True
    return False
def k3StartDeltaTypeConditionsMet(rootsys:rootSystem,index:int) -> bool:
    if(index == 0):
        return False
    deltaWords = rootsys.getWords(rootsys.delta)
    endings = np.array([rootsys.costfac(i)[1].weights for i in deltaWords[:index]])
    indexWeight = np.array(rootsys.costfac(deltaWords[index])[1].weights)
    for end in endings:
        if(sum(indexWeight)-1 == sum(end) and sum(indexWeight - end) == 1):
            return False
    for i in range(len(endings)):
        if(endings[i][-1] == 1):
            endings[i] = rootsys.delta - endings[i]
    indexWeight[deltaWords[index][-1].rootIndex] -= 1
    if(indexWeight[-1] == 1):
        indexWeight = rootsys.delta - indexWeight
    np.append(endings,[indexWeight],axis=0)
    return np.linalg.matrix_rank(endings) == index
def checkDeltaTypePrediction(rootsys:rootSystem,k=2):
    result = GenerateDeltaTypes(rootsys,k)
    deltaWords = rootsys.getWords(rootsys.delta)
    for i in range(len(result.deltaTypes)):
        if((result.deltaTypes[i].type == "cofac") != costfacDeltaTypeConditionsMet(rootsys,deltaWords,i)):
            yield (rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"cofac")
        #if((result.deltaTypes[i].type == "last") != lastSplitDeltaTypeConditionsMet(rootsys,deltaWords,i)):
        #    exceptions.append((rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"last"))
        #if((result.deltaTypes[i].type == "k3start") != k3StartDeltaTypeConditionsMet(rootsys,i)):
        #   yield (rootsys.type, str(rootsys.ordering),i+1,result.deltaTypes[i].type,"k3start") 
def checkDeltaTypePrectionPerms(rootsystems,k=2):
    for rootsys in rootsystems:
        for i in checkDeltaTypePrediction(rootsys,k):
            yield i

In [254]:
def writeDeltaTypesToFile(rootSystems,file="out.txt",notOnlyAllLast=True):
    fileobj = open(file,'w')
    fileobj.write("imaginary index, bracketing, splitting type, inserted imaginary index, left factor, right factor\n")
    for i in rootSystems:
        deltaTypesCol = GenerateDeltaTypes(i)
        if(notOnlyAllLast):
            if(not deltaTypesCol.notAllLast()):
                continue
        fileobj.write(deltaTypesCol.toCsv())
    fileobj.close()

### SL Word Algorithm

In [255]:
def checkSLWordAlgoPerms(rootsyss):
    for rootsys in rootsyss:
        for exc in checkSLWordAlgo(rootsys):
            yield (str(rootsys.ordering),exc,rootsys.parseToDeltaFormat(rootsys.getWords(exc)[0]),rootsys.parseToDeltaFormat(rootsys.SLWordAlgo(exc)))
def checkSLWordAlgo(rootsys:rootSystem):
    for k in range(rootsys.k):
        for baseWeight in rootsys.baseWeights[:-1]:
            weight = baseWeight + k*rootsys.delta
            if(word.letterListCmp(rootsys.getWords(weight)[0],rootsys.SLWordAlgo(weight).string) != 0):
                yield weight 
def checkGreedyAlgorithmForWord(rootsys:rootSystem, wrd:word) -> bool:
    return rootsys.SLWordAlgo(wrd,rootsys.SLWordAlgo(wrd.weights))

In [256]:
def checkHDeltasRepeat(perms):
    excepts = []
    for i in perms:
        hs = [j.hs for j in i.getAffineWords(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [257]:
def compareRootSystems(r1,r2):
    words = list(r1.weightToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.getWords(i[j].weights)[j])):
                yield (i[j].weights,str(i[j]),str(r2.getWords(i[j].weights)[j]))

In [258]:
def generateEquivalence(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compareRootSystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

In [259]:
def getMaxPeriodicity(rootSystems):
    max = 0
    maxOrdering = ""
    maxRoot = []
    for rootsys in rootSystems:
        for simpleRoot in rootsys.baseWeights[:-1]:
            result = rootsys.getPeriodicity(simpleRoot)
            if(result > max):
                max = result
                maxOrdering = str(rootsys.ordering)
                maxRoot=simpleRoot
    return (max,maxOrdering,maxRoot)

## Type A

In [238]:
Aperms = genPermutations('A',4,2)

In [183]:
A5 = rootSystem([0,1,2,3,4],'A',3)
print(A5.getWords([0,1,1,1,0])[0].noCommas())
print(*A5.SLWordAlgo([0,1,1,1,0]))

123
1 2 3


In [239]:
for i in checkSLWordAlgoPerms(Aperms):
    print(i)

  0%|          | 0/120 [00:00<?, ?it/s]

('0<1<3<2<4', array([1, 1, 1, 2, 1]), '043123', '043213')
('0<1<3<2<4', array([1, 1, 1, 1, 2]), '041234', '043214')
('0<1<3<2<4', array([1, 1, 1, 2, 2]), '0431234', '0432143')
('0<1<3<2<4', array([1, 1, 2, 2, 2]), '04321234', '04321432')
('0<1<3<2<4', array([1, 2, 2, 2, 2]), '043212341', '043214321')
('0<1<2<4<3', array([1, 1, 1, 2, 1]), '043123', '043213')
('0<1<2<4<3', array([1, 1, 1, 1, 2]), '041234', '043214')
('0<1<2<4<3', array([1, 1, 2, 2, 1]), '0432123', '0432132')
('0<1<2<4<3', array([1, 1, 1, 2, 2]), '0431234', '0432134')
('0<1<2<4<3', array([1, 2, 2, 2, 1]), '04321231', '04321321')
('0<1<2<4<3', array([1, 1, 2, 2, 2]), '04321234', '04321342')
('0<1<2<4<3', array([1, 2, 2, 2, 2]), '043212341', '043213421')
('0<1<2<3<4', array([1, 1, 1, 2, 1]), '043123', '043213')
('0<1<2<3<4', array([1, 1, 1, 1, 2]), '041234', '043214')
('0<1<2<3<4', array([1, 1, 2, 2, 1]), '0432123', '0432132')
('0<1<2<3<4', array([1, 1, 1, 2, 2]), '0431234', '0432143')
('0<1<2<3<4', array([1, 2, 2, 2, 1]), 

In [539]:
for i in checkDeltaTypePrectionPerms(Aperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [20]:
writeDeltaTypesToFile(Aperms,"ADeltaTypes.csv")

  0%|          | 0/720 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [32]:
getMaxPeriodicity(Aperms)

(3, '0<2<1<3<4<5', array([1, 1, 1, 0, 0, 0]))

## Type C

In [240]:
Cperms = genPermutations('C',5,2)

In [241]:
for i in checkSLWordAlgoPerms(Cperms):
    print(i)

  0%|          | 0/720 [00:00<?, ?it/s]

('0<1<2<3<4<5', array([1, 2, 2, 3, 2, 1]), '01234543123', '01234543213')
('0<1<2<3<4<5', array([1, 2, 2, 2, 3, 1]), '01234541234', '01234543214')
('0<1<2<3<4<5', array([1, 2, 2, 2, 2, 2]), '01234512345', '01234543215')
('0<1<2<3<4<5', array([1, 2, 2, 2, 3, 2]), '012345412345', '012345432154')
('0<1<2<3<4<5', array([1, 2, 3, 3, 2, 1]), '012345432123', '012345432132')
('0<1<2<3<4<5', array([1, 2, 2, 3, 3, 1]), '012345431234', '012345432143')
('0<1<2<3<4<5', array([1, 2, 2, 3, 3, 2]), '0123454312345', '0123454321543')
('0<1<2<3<4<5', array([1, 3, 3, 3, 2, 1]), '0123454321231', '0123454321321')
('0<1<2<3<4<5', array([1, 2, 3, 3, 3, 1]), '0123454321234', '0123454321432')
('0<1<2<3<4<5', array([1, 2, 2, 2, 4, 2]), '0123454123454', '0123454321544')
('0<1<2<3<4<5', array([1, 2, 3, 3, 3, 2]), '01234543212345', '01234543215432')
('0<1<2<3<4<5', array([1, 2, 2, 3, 4, 2]), '01234543123454', '01234543215443')
('0<1<2<3<4<5', array([1, 3, 3, 3, 3, 1]), '01234543212341', '01234543214321')
('0<1<2<3<4

In [543]:
for i in checkDeltaTypePrectionPerms(Cperms):
    print(i)

  0%|          | 0/720 [00:00<?, ?it/s]

In [22]:
writeDeltaTypesToFile(Cperms,"CDeltaTypes.csv")

  0%|          | 0/720 [00:00<?, ?it/s]

In [21]:
print(*[f"{str(i[0])}: {i[1]}\n" for i in checkMonotoneIncreasingImpCond(Cperms)])

5<4<3<2<1<0: [0 0 0 0 0 1]
 5<4<3<2<1<0: [1 0 0 0 0 1]
 5<4<3<2<1<0: [1 1 0 0 0 1]
 5<4<3<2<1<0: [2 0 0 0 0 1]
 5<4<3<2<1<0: [2 1 0 0 0 1]
 5<4<3<2<1<0: [2 2 0 0 0 1]



In [22]:
checkCondImpMonotoneIncreasing(Cperms)

[('1<3<0<2<4<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<2<5<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<4<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<0<5<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<0<5<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<4<5<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<2<4', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<0<4<2', '1,2,0,3,4,5,4,3,2'),
 ('1<3<5<4<0<2', '1,2,0,3,4,5,4,3,2'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<3<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<2<5<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<2<5', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<3<5<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<2<3', '1,2,3,0,4,5,4,3,2'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3'),
 ('1<4<0<5<3<2', '1,2,3,0,4,5,4,3,2'),
 ('1<4<3<0<2<5', '1,2,0,3,4,5,4,3,2'),
 ('1<4<3<0<5<2', '1,2,0,3,4,5,4,3,2')

In [25]:
checkHDeltasRepeat(Cperms)

[]

In [26]:
index=250
print(Cperms[index].ordering)
[print(*list(Cperms[index].costfac(i))+[i.hs],sep='-',) for i in Cperms[index].getAffineWords(Cperms[index].delta)]

2<0<3<5<1<4
2,1,3,4,0-2,1,3,4,5-[1 1 1 1 1]
2,1,3,4-2,1,3,4,5,0-[1 1 1 1 0]
2,1,3,0-2,1,3,4,5,4-[1 1 1 2 1]
2,1,0-2,1,3,4,5,4,3-[1 1 2 2 1]
2,3,4,5,4,3-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,4-[0 0 0 1 0]
2,1,0,2,1,3,4,5,4-2,1,3,0,2,1,3,4,5,4,2,1,3,0,2,1,3,4,5,4,3-[0 0 1 0 0]
2,3,4,5,4,3,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5-2,1,0,1-[0 1 2 2 1]
2,1,3,4,0-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,5-[1 1 1 1 1]
2,1,3,4,2,1,3,4,5-2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,2,1,3,4,0,2,1,3,4,5,0-[2 2 2 2 1]
2,1,3,0,2,1,3,4,5-2,1,3,

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [27]:
for i in generateEquivalence(Cperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<0<4<5 

In [86]:
writeDeltaTypesToFile(Cperms)

## Type G

In [58]:
G2perms = genPermutations('G',2,2)

In [59]:
for i in checkSLWordAlgoPerms(G2perms):
    print(i)

  0%|          | 0/6 [00:00<?, ?it/s]

In [545]:
for i in checkDeltaTypePrectionPerms(G2perms):
    print(i)

  0%|          | 0/6 [00:00<?, ?it/s]

In [572]:
writeDeltaTypesToFile(G2perms,"GDeltaTypes.csv",False)

  0%|          | 0/6 [00:00<?, ?it/s]

In [144]:
checkCondImpMonotoneIncreasing(G2perms)

[('1<0<2', '1,2,2,2'), ('1<0<2', '1,2,2,2,0')]

In [145]:
checkMonotoneIncreasingImpCond(G2perms)

[]

In [146]:
for i in G2perms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [148]:
checkHDeltasRepeat(G2perms)

[]

In [149]:
for i in generateEquivalence(G2perms):
    print(f"{i[0]} = {i[1]}")

## Type B

In [23]:
Bperms = genPermutations('B',6,4)

In [86]:
for i in checkSLWordAlgoPerms(Bperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [39]:
for i in checkSLWordAlgoPerms(Bperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

ValueError: Algorithm only word if the smallest occurs once

In [547]:
for i in checkDeltaTypePrectionPerms(Bperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [24]:
writeDeltaTypesToFile(Bperms,"BDeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [114]:
checkCondImpMonotoneIncreasing(Bperms)

[('2<0<1<3<4<5', '2,3,4,5,5,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<4<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,1,0'),
 ('2<0<1<3<5<4', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<3<5', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,1,0'),
 ('2<0<1<4<5<3', '2,3,4,5,5,4,3,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,1'),
 ('2<0<1<5<3<4', '2,3,4,5,5,1,0'),
 ('2<0<1<5<3<4', '2,3,4,5,5,4,3,1

In [22]:
checkMonotoneIncreasingImpCond(Bperms)

[]

In [116]:
for i in Bperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [118]:
checkHDeltasRepeat(Bperms)

[]

In [119]:
for i in generateEquivalence(Bperms):
    print(f"{i[0]} = {i[1]}")

0<1<3<4<5<2 = 0<1<3<4<2<5
0<1<4<3<5<2 = 0<1<4<3<2<5
0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<4<5<1<3 = 0<2<4<1<5<3
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<1<4<5<2 = 0<3<1<4<2<5
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<1<5<2 = 0<3<4<1<2<5
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<4<5<1<2 = 0<3<4<1<2<5
0<3<4<5<1<2 = 0<3<4<1<5<2
0<3<4<5<2<1 = 0<3<4<2<1<5
0<3<4<5<2<1 = 0<3<4<2<5<1
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<1<3<5<2 = 0<4<1<3<2<5
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<2<5<1<3 = 0<4<2<1<5<3
0<4<3<1<5<2 = 0<4<3<1<2<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<4<3<5<1<2 = 0<4<3<1<2<5
0<4<3<5<1<2 = 0<4<3<1<5<2
0<4<3<5<2<1 = 0<4<3<2<1<5
0<4<3<5<2<1 = 0<4<3<2<5<1
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<0<3<4<5<2 = 1<0<3<4<2<5
1<0<4<3<5<2 = 1<0<4<3<2<5
1<2<3<4<0<5 

## Type D

In [25]:
Dperms = genPermutations('D',6,4)

In [62]:
for i in checkSLWordAlgoPerms(Dperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [25]:
checkDeltaTypePrectionPerms(Dperms)

  0%|          | 0/40320 [00:00<?, ?it/s]

[]

In [26]:
writeDeltaTypesToFile(Dperms,"DDeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [130]:
checkMonotoneIncreasingImpCond(Dperms)

[('5<4<3<2<1<0', array([0, 0, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([0, 1, 0, 0, 0, 1])),
 ('5<4<3<2<1<0', array([1, 1, 0, 0, 0, 1]))]

In [131]:
checkCondImpMonotoneIncreasing(Dperms)

[('2<0<4<5<1<3', '2,3,1,5,4,3'),
 ('2<0<4<5<1<3', '2,3,1,5,4,3,0'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3'),
 ('2<0<5<4<1<3', '2,3,1,4,5,3,0'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3'),
 ('2<1<4<5<0<3', '2,3,0,5,4,3,1'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3'),
 ('2<1<5<4<0<3', '2,3,0,4,5,3,1'),
 ('2<4<0<5<1<3', '2,3,1,5,4,3'),
 ('2<4<0<5<1<3', '2,3,1,5,0,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,4,3'),
 ('2<4<1<5<0<3', '2,3,0,5,1,4,3'),
 ('2<4<5<0<1<3', '2,3,1,5,4,3'),
 ('2<4<5<0<1<3', '2,3,1,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,5,4,3'),
 ('2<4<5<1<0<3', '2,3,0,1,5,4,3'),
 ('2<5<0<4<1<3', '2,3,1,4,5,3'),
 ('2<5<0<4<1<3', '2,3,1,4,0,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,5,3'),
 ('2<5<1<4<0<3', '2,3,0,4,1,5,3'),
 ('2<5<4<0<1<3', '2,3,1,4,5,3'),
 ('2<5<4<0<1<3', '2,3,1,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,4,5,3'),
 ('2<5<4<1<0<3', '2,3,0,1,4,5,3'),
 ('3<0<1<4<5<2', '3,2,5,1,0,2'),
 ('3<0<1<4<5<2', '3,2,5,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,1,0,2'),
 ('3<0<1<5<4<2', '3,2,4,5,1,0,2'),
 ('3<0<4<1<5<2', '3,2,5,1,0,2'),
 ('3<0<4<1<5<2'

In [133]:
for i in Dperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [134]:
checkHDeltasRepeat(Dperms)

[]

In [135]:
for i in generateEquivalence(Dperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1<5 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<1<4<5
0<2<3<4<5<1 = 0<2<3<4<1<5
0<2<3<5<1<4 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<1<5<4
0<2<3<5<4<1 = 0<2<3<5<1<4
0<2<4<3<5<1 = 0<2<4<3<1<5
0<2<5<3<4<1 = 0<2<5<3<1<4
0<3<2<4<1<5 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<1<4<5
0<3<2<4<5<1 = 0<3<2<4<1<5
0<3<2<5<1<4 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<1<5<4
0<3<2<5<4<1 = 0<3<2<5<1<4
0<3<4<2<5<1 = 0<3<4<2<1<5
0<3<5<2<4<1 = 0<3<5<2<1<4
0<4<2<3<5<1 = 0<4<2<3<1<5
0<4<3<2<5<1 = 0<4<3<2<1<5
0<5<2<3<4<1 = 0<5<2<3<1<4
0<5<3<2<4<1 = 0<5<3<2<1<4
1<2<3<4<0<5 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<0<4<5
1<2<3<4<5<0 = 1<2<3<4<0<5
1<2<3<5<0<4 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<0<5<4
1<2<3<5<4<0 = 1<2<3<5<0<4
1<2<4<3<5<0 = 1<2<4<3<0<5
1<2<5<3<4<0 = 1<2<5<3<0<4
1<3<2<4<0<5 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<0<4<5
1<3<2<4<5<0 = 1<3<2<4<0<5
1<3<2<5<0<4 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<0<5<4
1<3<2<5<4<0 = 1<3<2<5<0<4
1<3<4<2<5<0 = 1<3<4<2<0<5
1<3<5<2<4<0 = 1<3<5<2<0<4
1<4<2<3<5<0 = 1<4<2<3<0<5
1<4<3<2<5<0 = 1<4<3<2<0<5
1<5<2<3<4<0 

## Type E

In [27]:
Eperms = genPermutations('E',6,4)

In [61]:
for i in checkSLWordAlgoPerms(Eperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

In [393]:
for i in checkDeltaTypePrectionPerms(Eperms):
    print(i)

  0%|          | 0/5040 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [21]:
checkHDeltasRepeat(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

[]

In [40]:
for i in Eperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [28]:
writeDeltaTypesToFile(Eperms,"E6DeltaTypes.csv")

  0%|          | 0/5040 [00:00<?, ?it/s]

In [46]:
for i in generateEquivalence(Eperms):
    print(f"{i[0]} = {i[1]}")

0<1<2<4<5<6<3 = 0<1<2<4<5<3<6
0<1<2<5<4<6<3 = 0<1<2<5<4<3<6
0<1<3<4<5<6<2 = 0<1<3<4<5<2<6
0<1<3<5<4<6<2 = 0<1<3<5<4<2<6
0<1<4<2<5<6<3 = 0<1<4<2<5<3<6
0<1<4<3<5<6<2 = 0<1<4<3<5<2<6
0<1<4<5<2<6<3 = 0<1<4<5<2<3<6
0<1<4<5<3<6<2 = 0<1<4<5<3<2<6
0<1<4<5<6<2<3 = 0<1<4<5<2<3<6
0<1<4<5<6<2<3 = 0<1<4<5<2<6<3
0<1<4<5<6<3<2 = 0<1<4<5<3<2<6
0<1<4<5<6<3<2 = 0<1<4<5<3<6<2
0<1<5<2<4<6<3 = 0<1<5<2<4<3<6
0<1<5<3<4<6<2 = 0<1<5<3<4<2<6
0<1<5<4<2<6<3 = 0<1<5<4<2<3<6
0<1<5<4<3<6<2 = 0<1<5<4<3<2<6
0<1<5<4<6<2<3 = 0<1<5<4<2<3<6
0<1<5<4<6<2<3 = 0<1<5<4<2<6<3
0<1<5<4<6<3<2 = 0<1<5<4<3<2<6
0<1<5<4<6<3<2 = 0<1<5<4<3<6<2
0<2<1<4<5<6<3 = 0<2<1<4<5<3<6
0<2<1<5<4<6<3 = 0<2<1<5<4<3<6
0<2<3<4<5<1<6 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<1<5<6
0<2<3<4<5<6<1 = 0<2<3<4<5<1<6
0<2<3<4<6<1<5 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<1<6<5
0<2<3<4<6<5<1 = 0<2<3<4<6<1<5
0<2<3<5<4<6<1 = 0<2<3<5<4<1<6
0<2<3<5<6<1<4 = 0<2<3<5<1<6<4
0<2<3<6<4<5<1 = 0<2<3<6<4<1<5
0<2<4<1<5<6<3 = 0<2<4<1<5<3<6
0<2<4<3<5<1<6 = 0<2<4<3<1<5<6
0<2<4<3<5<

KeyboardInterrupt: 

In [19]:
getMaxPeriodicity(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

(10, '2<0<1<4<6<5<3', array([1, 0, 1, 1, 1, 1, 0]))

## Type F

In [260]:
Fperms = genPermutations('F',4,3)

In [261]:
for i in checkSLWordAlgoPerms(Fperms):
    print(i)

  0%|          | 0/120 [00:00<?, ?it/s]

('0<1<2<4<3', array([1, 2, 3, 5, 2]), '0123344233123', '0123344233213')
('0<1<2<4<3', array([1, 2, 3, 4, 3]), '0123344231234', '0123344233214')
('0<1<2<4<3', array([1, 2, 3, 5, 3]), '01233442331234', '01233442332134')
('0<1<2<4<3', array([1, 2, 4, 5, 2]), '01233442332123', '01233442332132')
('0<1<2<4<3', array([1, 2, 4, 5, 3]), '012334423321234', '012334423321342')
('0<1<2<4<3', array([1, 2, 4, 6, 2]), '012334423321233', '012334423321323')
('0<1<2<4<3', array([1, 3, 4, 5, 2]), '012334423321231', '012334423321321')
('0<1<2<4<3', array([1, 2, 4, 6, 3]), '0123344233212334', '0123344233213423')
('0<1<2<4<3', array([1, 3, 4, 5, 3]), '0123344233212341', '0123344233213421')
('0<1<2<4<3', array([1, 3, 4, 6, 2]), '0123344233212331', '0123344233213231')
('0<1<2<4<3', array([1, 2, 4, 6, 4]), '01233442332123344', '01233442332134234')
('0<1<2<4<3', array([1, 3, 4, 6, 3]), '01233442332123341', '01233442332134231')
('0<1<2<4<3', array([1, 3, 5, 6, 2]), '01233442332123312', '01233442332132312')
('0<1<

In [574]:
for i in checkDeltaTypePrectionPerms(Fperms):
    print(i)

  0%|          | 0/120 [00:00<?, ?it/s]

('F', '2<0<1<4<3', 2, 'lastSmallest', 'cofac')
('F', '2<0<4<1<3', 2, 'lastSmallest', 'cofac')
('F', '2<1<0<4<3', 2, 'lastSmallest', 'cofac')
('F', '2<1<4<0<3', 2, 'lastSmallest', 'cofac')
('F', '2<1<4<3<0', 2, 'lastSmallest', 'cofac')
('F', '2<4<0<1<3', 3, 'lastSmallest', 'cofac')
('F', '2<4<1<0<3', 3, 'lastSmallest', 'cofac')
('F', '2<4<1<3<0', 3, 'lastSmallest', 'cofac')
('F', '3<0<1<2<4', 2, 'lastSmallest', 'cofac')
('F', '3<0<2<1<4', 2, 'lastSmallest', 'cofac')
('F', '3<0<2<4<1', 2, 'lastSmallest', 'cofac')
('F', '3<1<0<2<4', 2, 'lastSmallest', 'cofac')
('F', '3<1<2<0<4', 2, 'lastSmallest', 'cofac')
('F', '3<1<2<4<0', 2, 'lastSmallest', 'cofac')
('F', '3<2<0<1<4', 2, 'lastSmallest', 'cofac')
('F', '3<2<1<0<4', 2, 'lastSmallest', 'cofac')
('F', '3<2<0<4<1', 2, 'lastSmallest', 'cofac')
('F', '3<2<1<4<0', 2, 'lastSmallest', 'cofac')
('F', '3<2<4<1<0', 2, 'lastSmallest', 'cofac')
('F', '3<2<4<0<1', 2, 'lastSmallest', 'cofac')


In [30]:
writeDeltaTypesToFile(Fperms,"FDeltaTypes.csv",notOnlyAllLast=False)

  0%|          | 0/120 [00:00<?, ?it/s]

In [22]:
checkMonotoneIncreasingImpCond(Fperms)

UnboundLocalError: cannot access local variable 'returnArr' where it is not associated with a value

In [ ]:
checkCondImpMonotoneIncreasing(Fperms)

[('1<0<2<3<4', '1,2,3,4,3,4,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<2<3<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,0'),
 ('1<0<2<4<3', '1,2,3,3,4,4,2,3,3,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,0'),
 ('1<0<3<2<4', '1,2,3,4,3,4,2,3,3,2,0'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,3'),
 ('1<0<3<4<2', '1,2,3,4,3,2,4,0'),
 ('1<0<3<

In [ ]:
for i in Fperms:
   ex = i.checkConvexity()
   if(len(ex) != 0):
       print("Convexity doesn't hold")

In [ ]:
checkHDeltasRepeat(Fperms)

[]

In [ ]:
getMaxPeriodicity(Fperms)

(11, '1<0<3<4<2', array([0, 1, 2, 2, 0]))

In [ ]:
for i in generateEquivalence(Fperms):
    print(f"{i[0]} = {i[1]}")

0<2<3<4<1 = 0<2<3<1<4
0<3<2<4<1 = 0<3<2<1<4
1<2<3<0<4 = 1<2<0<3<4
1<2<3<4<0 = 1<2<0<3<4
1<2<3<4<0 = 1<2<3<0<4
1<2<4<0<3 = 1<2<0<4<3
1<2<4<3<0 = 1<2<0<4<3
1<2<4<3<0 = 1<2<4<0<3
1<3<2<4<0 = 1<3<2<0<4
1<3<4<0<2 = 1<3<0<4<2
1<4<2<3<0 = 1<4<2<0<3
2<0<3<4<1 = 2<0<3<1<4
2<1<3<0<4 = 2<1<0<3<4
2<1<3<4<0 = 2<1<0<3<4
2<1<3<4<0 = 2<1<3<0<4
2<1<4<0<3 = 2<1<0<4<3
2<1<4<3<0 = 2<1<0<4<3
2<1<4<3<0 = 2<1<4<0<3
2<3<0<4<1 = 2<3<0<1<4
2<3<1<4<0 = 2<3<1<0<4
2<3<4<0<1 = 2<3<0<1<4
2<3<4<0<1 = 2<3<0<4<1
2<3<4<1<0 = 2<3<1<0<4
2<3<4<1<0 = 2<3<1<4<0
2<4<1<3<0 = 2<4<1<0<3
3<0<2<4<1 = 3<0<2<1<4
3<1<2<4<0 = 3<1<2<0<4
3<1<4<0<2 = 3<1<0<4<2
3<2<0<4<1 = 3<2<0<1<4
3<2<1<4<0 = 3<2<1<0<4
3<2<4<0<1 = 3<2<0<1<4
3<2<4<0<1 = 3<2<0<4<1
3<2<4<1<0 = 3<2<1<0<4
3<2<4<1<0 = 3<2<1<4<0
4<1<2<3<0 = 4<1<2<0<3
4<2<1<3<0 = 4<2<1<0<3
